In [8]:
import os
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from dotenv import find_dotenv, load_dotenv
from langchain.llms import OpenAI
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
import re
import pandas as pd
import logging
from summarizer import Summarizer
import pprint


logging.basicConfig(format="%(asctime)s - %(levelname)s - %(message)s", level=logging.INFO)
logger = logging.getLogger(__name__)


load_dotenv(find_dotenv())
query_memory = []


CHUNK_SIZE = 2000
CHUNK_OVERLAP = 1000
TEMPERATURE = 0
k = 5


PROMPT_TEMPLATE_MODEL = PromptTemplate(
    input_variables=["question", "docs"],
    template="""
    As an AI assistant, my role is to meticulously analyze court transcripts and extract information about law enforcement personnel.
    The names of law enforcement personnel will be prefixed by one of the following titles: officer, detective, deputy, lieutenant, 
    sergeant, captain, officer, coroner, investigator, criminalist, patrolman, or technician.

    Query: {question}

    Transcripts: {docs}

    The response will contain:

    1) The name of a officer, detective, deputy, lieutenant, 
       sergeant, captain, officer, coroner, investigator, criminalist, patrolman, or technician - 
       if an individual's name is not associated with one of these titles they do not work in law enforcement.
       Please prefix the name with "Officer Name: ". 
       For example, "Officer Name: John Smith".

    2) If available, provide an in-depth description of the context of their mention. 
       If the context induces ambiguity regarding the individual's employment in law enforcement, 
       remove the individual.
       Please prefix this information with "Officer Context: ". 

    Continue this pattern of identifying persons, until all law enforcement personnel are identified.  

    Additional guidelines for the AI assistant:
    - Titles may be abbreviated to the following Sgt., Cpl, Cpt, Det., Ofc., Lt., P.O. and P/O
    - Titles "Technician" and "Tech" might be used interchangeably.
    - Derive responses from factual information found within the police reports.
    - If the context of an identified person's mention is not clear in the report, provide their name and note that the context is not specified.
    - Do not extract information about victims and witnesses
""",
)


PROMPT_TEMPLATE_HYDE = PromptTemplate(input_variables=["question"], template="""
    You're an AI assistant specializing in criminal justice research. 
    Your main focus is on identifying the names and providing detailed context of mention for each law enforcement personnel. 
    This includes police officers, detectives, deupties, lieutenants, sergeants, captains, technicians, coroners, investigators, patrolman, and criminalists, 
    as described in court transcripts.
    Be aware that the titles "Detective" and "Officer" might be used interchangeably.
    Be aware that the titles "Technician" and "Tech" might be used interchangeably.

    Question: {question}

    Roles and Responses:""")


def clean_name(officer_name):
    return re.sub(
        r"(Detective|Officer|Deputy|Captain|[CcPpLl]|Sergeant|Lieutenant|Techn?i?c?i?a?n?|Investigator)\.?\s+",
        "",
        officer_name,
    )


def extract_officer_data(formatted_response):
    officer_data = []
    response_lines = formatted_response.split("\n")

    for line in response_lines:
        if line.startswith("Officer Name"):
            officer_name = line.split(":", 1)[1].strip()
            officer_title = re.search(
                r"(Detective|Officer|Deputy|Captain|[CcPpLl]|Sergeant|Lieutenant|Techn?i?c?i?a?n?|Investigator)\.?",
                officer_name,
            )
            if officer_title:
                officer_title = officer_title.group()
            else:
                officer_title = ""
            officer_name = clean_name(officer_name)
        elif line.startswith("Officer Context"):
            split_line = line.split(":", 1)
            if len(split_line) > 1:
                officer_context = split_line[1].strip()
            else:
                officer_context = ""  
            officer_data.append(
                {
                    "Officer Name": officer_name,
                    "Officer Context": officer_context,
                    "Officer Title": officer_title,
                }
            )

    return officer_data


def generate_hypothetical_embeddings():
    llm = OpenAI()
    prompt = PROMPT_TEMPLATE_HYDE

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    base_embeddings = OpenAIEmbeddings()

    embeddings = HypotheticalDocumentEmbedder(
        llm_chain=llm_chain, base_embeddings=base_embeddings
    )
    return base_embeddings


def get_response_from_query(db, query, temperature, k):
    logger.info("Performing query...")
    docs = db.similarity_search(query, k=k)
    pprint.pprint(docs)
    docs_page_content = " ".join([d.page_content for d in docs])
    pprint.pprint(docs_page_content)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo-0613")

    prompt = PROMPT_TEMPLATE_MODEL

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(question=query, docs=docs_page_content, temperature=temperature)

    formatted_response = ""
    officers = response.split("Officer Name:")
    for i, officer in enumerate(officers):
        if officer.strip() != "":
            formatted_response += f"Officer Name {i}:{officer.replace('Officer Context:', 'Officer Context ' + str(i) + ':')}\n\n"

    officer_data = extract_officer_data(formatted_response)
    return officer_data, docs


# QUERIES = [
#     "Identify individuals, by name, with the specific titles of police officers, sergeants, lieutenants, captains, detectives, homicide officers, and crime lab personnel in the transcript. Provide the context of their mention, if available.",
#     "List individuals, by name, specifically titled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel mentioned in the transcript and provide the context of their mention.",
#     "Locate individuals, by name, specifically referred to as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and explain their context of mention.",
#     "Highlight individuals, by name, explicitly titled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and describe their context of mention.",
#     "Outline individuals, by name, directly identified as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and specify their context of mention.",
#     "Pinpoint individuals, by name, directly labeled as police officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript and provide their context of mention.",
# ]


QUERIES = [
    "Identify individuals, by name, with the specific titles of officers, sergeants, lieutenants, captains, detectives, homicide officers, and crime lab personnel in the transcript. Specifically, provide the context of their mention related to key events in the case, if available.",
    "List individuals, by name, directly titled as officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel mentioned in the transcript. Provide the context of their mention in terms of any significant decisions they made or actions they took.",
    "Locate individuals, by name, directly referred to as officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript. Explain the context of their mention in relation to their interactions with other individuals in the case.",
    "Highlight individuals, by name, directly titled as officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript. Describe the context of their mention, specifically noting any roles or responsibilities they held in the case.",
    "Outline individuals, by name, directly identified as officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript. Specify the context of their mention in terms of any noteworthy outcomes or results they achieved.",
    "Pinpoint individuals, by name, directly labeled as officers, sergeants, lieutenants, captains, detectives, homicide units, and crime lab personnel in the transcript. Provide the context of their mention, particularly emphasizing any significant incidents or episodes they were involved in.",

]


def summarize_context(context):
    model = Summarizer()
    result = model(context, min_length=60)
    summary = "".join(result)
    return summary


def preprocess_single_document(file_path, embeddings):
    logger.info(f"Processing Word document: {file_path}")

    loader = Docx2txtLoader(file_path)
    text = loader.load()
    logger.info(f"Text loaded from Word document: {file_path}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    docs = text_splitter.split_documents(text)

    db = FAISS.from_documents(docs, embeddings)

    return db


def process_query(embeddings):
    doc_directory = "../../data/convictions/transcripts"

    for file_name in os.listdir(doc_directory):
        if file_name.endswith(".docx"):
            csv_output_path = os.path.join(doc_directory, f"{file_name}.csv")
            if os.path.exists(csv_output_path):
                logger.info(f"CSV output for {file_name} already exists. Skipping...")
                continue

            file_path = os.path.join(doc_directory, file_name)
            output_data = []

            db = preprocess_single_document(file_path, embeddings)
            for query in QUERIES:
                officer_data, _ = get_response_from_query(db, query, TEMPERATURE, k)
                for item in officer_data:
                    item["Query"] = query
                    item["Prompt Template for Hyde"] = PROMPT_TEMPLATE_HYDE
                    item["Prompt Template for Model"] = PROMPT_TEMPLATE_MODEL
                    item["Chunk Size"] = CHUNK_SIZE
                    item["Chunk Overlap"] = CHUNK_OVERLAP
                    item["Temperature"] = TEMPERATURE
                    item["k"] = k
                    item["hyde"] = "1"
                output_data.extend(officer_data)

            output_df = pd.DataFrame(output_data)
            # output_df = output_df.drop_duplicates(subset=["of", "officer_context"])
            output_df.to_csv(csv_output_path, index=False)

def main():
    embeddings = generate_hypothetical_embeddings()
    process_query(embeddings)

if __name__ == "__main__":
    main()

2023-07-19 16:38:33,210 - INFO - Processing Word document: ../../data/convictions/transcripts\(C) Det. Martin Venezia Testimony - Trial One.docx
2023-07-19 16:38:33,262 - INFO - Text loaded from Word document: ../../data/convictions/transcripts\(C) Det. Martin Venezia Testimony - Trial One.docx
2023-07-19 16:38:38,369 - INFO - Performing query...


[Document(page_content="A\ncome up and same way we treat any\nother murders, if you don't solve it\ninitially and you run up all the\n9\n10\nleads that are available to you, you\n11\ncontinue on to your Job and if further\nleads develop on an old murder case\n1 5\nthat remains unsolved, then that comes\n11\nup as time allows and you follow that\nlead.\n1,5\n1 think you mentioned earlier that you get\na l guess what we call an unconfirmed\n1 -\n1\n19\nCl, or confidential informant report,\n15\napproximately four to five months\n-\n/ before you interviewed Mr. Adams, is\nthat correct?\n.. About that time, yes.\nA\nWhen you got that report, what, if any-\n.\nthing, did you do to confirm its\n.\n..\n-\nveracity?\nI looked into the background, as I recall,\n-\n.\n. when it came through, 1 looked into .\n.\nthe background of the individual sup-\nplying the Information first to see\n...\nif there was motive between he and\nMr. Adams to tell me this.\nLacking .\n- 58-\n: --\n1 .. .\nthat, I pu

2023-07-19 16:39:10,295 - WARNING - Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
2023-07-19 16:39:19,711 - INFO - Performing query...


[Document(page_content="...\nlarge as this area here, myself to the\nbench, and back around, It has a sink\n, and a large table in the middle:\nWho else was In the room with you at that\ntime ?\n-17-\nTwo inmates, Inmate Cannon and Inmate\n-\nA\nAnderson.\n2\nAnd who else? The defendant was there.\n5\nThe defendant, Mr. Adams, Sergeant Ruiz,\nA\nDetective Ursin and myself: --\nAfter the defendant was given his rights,\nQ\ndid you speak with him?\n.\ninitially, no, sir.\nA\n9\nAfter initially.\nQ\n.\nEventually did you speak\nwith him?\nA\nYes, sir, i did.\nWho spoke with him first?\n-\nQ\n15\nSergeant Ruiz.\nA\nAnd that was on an unrelated matter?\n13\nYes.\nA\nAfter Sergeant Ruiz talked to Mr. Adams ,\nQ\ndid you speak with him?\n13\nYes, sir, I did.\nA\n10\nAnd who was present? I assume this was in\nthe same conference room?\n:1\nWe took a break, Sergeant Ruiz finished,\nA\n..\ngot some coffee, went into the rest-\n- -\nroom and came to the conference room.\nWho was in the room at th

2023-07-19 16:39:36,200 - INFO - Performing query...


...

[Document(page_content="25\nCross-Examination by Mr. Dunn\n28\n17\nDETECTIVE MASON SPONG\n18\nDirect . Examination by Ms. Atkins\n29\n19\nCross-Examination by Mr. Dunn\n50\n20\nRedirect Examination by Ms. Atkins\n59\n21\nLINDA SANDRONI\n22\nDirect Examination by Ms. Atkins\n61\nCross-Examination by Mr. Dunn\n82\n21\nEXHIBIT INDEX\nSTATE EXHIBITS\nNo. 1\nPhotograph\n5\nNo.\nPhotograph\n2\n5\nNo.\nPhotograph\n3\n5\nPhotograph\nNo.\n4\n5\n28\nNo.\n5\nPhotograph\n6\nNo.\nPhotograph\n6\n6\n29\nNo.\nPhotograph\n7\n13\nNo. 8\nPhotograph\n13\n50\nNo.\nPhotograph\n9\n13\nNo. 10\nPhotograph\n13\n31\nNo. 11\nPhotograph\n14\n2\nAFTER BEING DULY SWORN BY THE\n1\nCOURT, OFFICER HARRY O'NEAL\nTESTIFIED AS FOLLOWS:\n2\nPlease be seated. State your name for the record.\n5\nBY THE WITNESS:\n1\nHarry O'Neal, 0-'-N-E-A-L.\n5\nDIRECT EXAMINATION\n6\n-\nEXAMINATION BY MS. SMITH:\nWhat is your occupation?\n00 G.\nI am a criminalist with the New Orleans Police\nA\nDepartment Crime Lab.\n10\nHow long have you 

2023-07-19 16:44:35,316 - INFO - Performing query...


[Document(page_content="25\nCross-Examination by Mr. Dunn\n28\n17\nDETECTIVE MASON SPONG\n18\nDirect . Examination by Ms. Atkins\n29\n19\nCross-Examination by Mr. Dunn\n50\n20\nRedirect Examination by Ms. Atkins\n59\n21\nLINDA SANDRONI\n22\nDirect Examination by Ms. Atkins\n61\nCross-Examination by Mr. Dunn\n82\n21\nEXHIBIT INDEX\nSTATE EXHIBITS\nNo. 1\nPhotograph\n5\nNo.\nPhotograph\n2\n5\nNo.\nPhotograph\n3\n5\nPhotograph\nNo.\n4\n5\n28\nNo.\n5\nPhotograph\n6\nNo.\nPhotograph\n6\n6\n29\nNo.\nPhotograph\n7\n13\nNo. 8\nPhotograph\n13\n50\nNo.\nPhotograph\n9\n13\nNo. 10\nPhotograph\n13\n31\nNo. 11\nPhotograph\n14\n2\nAFTER BEING DULY SWORN BY THE\n1\nCOURT, OFFICER HARRY O'NEAL\nTESTIFIED AS FOLLOWS:\n2\nPlease be seated. State your name for the record.\n5\nBY THE WITNESS:\n1\nHarry O'Neal, 0-'-N-E-A-L.\n5\nDIRECT EXAMINATION\n6\n-\nEXAMINATION BY MS. SMITH:\nWhat is your occupation?\n00 G.\nI am a criminalist with the New Orleans Police\nA\nDepartment Crime Lab.\n10\nHow long have you 

2023-07-19 16:44:42,810 - INFO - Performing query...


[Document(page_content="25\nCross-Examination by Mr. Dunn\n28\n17\nDETECTIVE MASON SPONG\n18\nDirect . Examination by Ms. Atkins\n29\n19\nCross-Examination by Mr. Dunn\n50\n20\nRedirect Examination by Ms. Atkins\n59\n21\nLINDA SANDRONI\n22\nDirect Examination by Ms. Atkins\n61\nCross-Examination by Mr. Dunn\n82\n21\nEXHIBIT INDEX\nSTATE EXHIBITS\nNo. 1\nPhotograph\n5\nNo.\nPhotograph\n2\n5\nNo.\nPhotograph\n3\n5\nPhotograph\nNo.\n4\n5\n28\nNo.\n5\nPhotograph\n6\nNo.\nPhotograph\n6\n6\n29\nNo.\nPhotograph\n7\n13\nNo. 8\nPhotograph\n13\n50\nNo.\nPhotograph\n9\n13\nNo. 10\nPhotograph\n13\n31\nNo. 11\nPhotograph\n14\n2\nAFTER BEING DULY SWORN BY THE\n1\nCOURT, OFFICER HARRY O'NEAL\nTESTIFIED AS FOLLOWS:\n2\nPlease be seated. State your name for the record.\n5\nBY THE WITNESS:\n1\nHarry O'Neal, 0-'-N-E-A-L.\n5\nDIRECT EXAMINATION\n6\n-\nEXAMINATION BY MS. SMITH:\nWhat is your occupation?\n00 G.\nI am a criminalist with the New Orleans Police\nA\nDepartment Crime Lab.\n10\nHow long have you 